# Tutorial 4: Saving Your Results

In [2]:
import ponder.bigquery
import modin.pandas as pd
import json; import os 
creds = json.load(open(os.path.expanduser("../../../doris-bigquery-381416-497e94001f15.json")))
bigquery_con = ponder.bigquery.connect(creds, schema = "TEST")
ponder.bigquery.init(bigquery_con)

2023-03-22 19:15:05,356 - INFO - Establishing connection to pushdown.ponder-internal.io



Connected to
       ___               __
      / _ \___  ___  ___/ /__ ____
     / ___/ _ \/ _ \/ _  / -_) __/
    /_/___\___/_//_/\_,_/\__/_/
      / __/__ _____  _____ ____
     _\ \/ -_) __/ |/ / -_) __/
    /___/\__/_/  |___/\__/_/



Let's say that you used Ponder to run some analysis and you want to store the results back to the database. In this tutorial, we will show how you can use the ``to_sql`` command to write your dataframe to table in your database. 

In [2]:
df = pd.read_sql("PONDER_CITIBIKE",con=bigquery_con)

RuntimeError: Error in getting table metadata SELECT * FROM TEST.`PONDER_CITIBIKE` LIMIT 1 because 404 Not found: Table innate-empire-381416:TEST.PONDER_CITIBIKE was not found in location US

Location: US
Job ID: 795d61a6-9375-4c20-a83f-8979d1cc617a


After connecting to our `PONDER_CITIBIKE` table, we see that there are a lot of records with missing values. So we drop these rows to clean up our dataset.

In [ ]:
df_cleaned = df.dropna()

In [ ]:
df_cleaned

Like in pandas, when you are working with a dataframe, you are always working on a temporary copy of the data. 

By default, all pandas operations returns a copy of the dataframe that was operated on. Ponder never make any modifications to your original data. This is especially important in the data warehouse context as the tables are often regarded as the “source of truth” and can be shared across many teams.

This also means that the reference to your dataframe df is only accessible throughout the session. That means that once you exit the session, the dataframe is no longer accessible. 

So if we plan to use the cleaned up data for our analysis later, we can persist it in a table via `to_sql`.

In [ ]:
df_cleaned.to_sql("PONDER_CITIBIKE_CLEANED",con=bigquery_con, index=False)

Now we can access new new table and continue our analysis whereever we want:

In [ ]:
pd.read_sql("PONDER_CITIBIKE_CLEANED","auto")